**Scenario 1: No information about the epitope is available.**

We have downloaded and saved the first model of each cluster (in the PDB format) from the result page: https://rascar.science.uu.nl/haddock2.4/result/4242424242/0-4G6M-Ab-Ag-surface-sas-filtered. These files can be downloaded from the github folder using the following code segment. We also placed the 4G6M-matched.pdb file in the same folder (which will be out reference structure).


In [1]:
!git clone https://github.com/computationalpharmaceutics/3FG005.git

Cloning into '3FG005'...
remote: Enumerating objects: 180, done.
remote: Total 180 (delta 0), reused 0 (delta 0), pack-reused 180 (from 1)
Receiving objects: 100% (180/180), 29.47 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (73/73), done.


We are not collecting all the PDB files for the calculation. You will see the name of the cluster PDB files below.

In [2]:
import glob


# Adjust the path to match your repo structure
pdb_folder = "/content/3FG005/Lab02_Docking/Scenario_1/"
pdb_files = sorted(glob.glob(pdb_folder + "*.pdb"))

ref_file = [f for f in pdb_files if "4G6M-matched" in f][0]
cluster_files = [f for f in pdb_files if "4G6M-matched" not in f]

print("Reference:", ref_file)
print("Cluster files:", cluster_files)

Reference: /content/3FG005/Lab02_Docking/Scenario_1/4G6M-matched.pdb
Cluster files: ['/content/3FG005/Lab02_Docking/Scenario_1/cluster10_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster13_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster15_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster21_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster22_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster2_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster5_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster6_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster7_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_1/cluster8_1.pdb']


You will now see the rmsd value for each molecules in the terminal screen when you run the following two code segments.

In CAPRI, the L-RMSD value defines the quality of a model:

incorrect model: L-RMSD>10Å

acceptable model: L-RMSD<10Å

medium quality model: L-RMSD<5Å

high quality model: L-RMSD<1Å


What is the quality of these models? Did any model pass the acceptable threshold?


In [3]:
import numpy as np
import os

def parse_pdb_backbone_chain_atoms(pdb_file, chain_id, resid_range=None):
    backbone_atoms = {'N', 'CA', 'C', 'O'}
    atoms = {}
    with open(pdb_file, 'r') as f:
        for line in f:
            if not line.startswith("ATOM"): continue
            atom_name = line[12:16].strip()
            chain = line[21].strip()
            resid = int(line[22:26])
            if atom_name not in backbone_atoms or chain != chain_id: continue
            if resid_range and not (resid_range[0] <= resid <= resid_range[1]): continue
            x, y, z = float(line[30:38]), float(line[38:46]), float(line[46:54])
            atoms[(resid, atom_name)] = np.array([x, y, z])
    return atoms

def kabsch(P, Q):
    C = np.dot(P.T, Q)
    V, S, W = np.linalg.svd(C)
    if np.linalg.det(V) * np.linalg.det(W) < 0: V[:, -1] = -V[:, -1]
    return np.dot(V, W)

def calculate_rmsd(P, Q):
    return np.sqrt(np.mean(np.sum((P - Q)**2, axis=1)))

In [4]:
# Parse reference once
ref_A = parse_pdb_backbone_chain_atoms(ref_file, 'A', resid_range=(0, 225))
ref_B = parse_pdb_backbone_chain_atoms(ref_file, 'B')

for mob_file in cluster_files:
    mob_A = parse_pdb_backbone_chain_atoms(mob_file, 'A', resid_range=(0, 225))
    mob_B = parse_pdb_backbone_chain_atoms(mob_file, 'B')

    # Match atom keys
    keys_A = sorted(set(ref_A) & set(mob_A))
    keys_B = sorted(set(ref_B) & set(mob_B))

    if len(keys_A) == 0 or len(keys_B) == 0:
        print(f"{os.path.basename(mob_file)}: No common atoms — skipping.")
        continue

    P_A = np.array([mob_A[k] for k in keys_A])
    Q_A = np.array([ref_A[k] for k in keys_A])
    P_B = np.array([mob_B[k] for k in keys_B])
    Q_B = np.array([ref_B[k] for k in keys_B])

    # Align on chain A
    P_A_c = P_A - P_A.mean(axis=0)
    Q_A_c = Q_A - Q_A.mean(axis=0)
    R = kabsch(P_A_c, Q_A_c)

    # Apply rotation to chain B and calculate RMSD
    P_B_c = P_B - P_B.mean(axis=0)
    Q_B_c = Q_B - Q_B.mean(axis=0)
    P_B_aligned = P_B_c @ R
    rmsd_val = calculate_rmsd(P_B_aligned, Q_B_c)

    print(f"RMSD of {os.path.basename(mob_file)}: {rmsd_val:.3f} Å")

RMSD of cluster10_1.pdb: 22.135 Å
RMSD of cluster13_1.pdb: 22.221 Å
RMSD of cluster15_1.pdb: 22.604 Å
RMSD of cluster21_1.pdb: 21.725 Å
RMSD of cluster22_1.pdb: 22.207 Å
RMSD of cluster2_1.pdb: 23.582 Å
RMSD of cluster5_1.pdb: 20.174 Å
RMSD of cluster6_1.pdb: 21.948 Å
RMSD of cluster7_1.pdb: 22.968 Å
RMSD of cluster8_1.pdb: 16.979 Å


Let’s now check if the active and passive residues which we defined are actually part of the interface. For this, we will first install py3Dmol

In [5]:
!pip install -q py3Dmol

The active and passive residues are as follows

In [6]:
ab_active = [26,27,28,29,30,31,32,55,56,57,101,102,103,106,108,146,147,148,150,151,152,170,172,212,213,214,215]
ag_passive = [22,46,47,48,64,71,72,73,74,75,82,84,85,86,87,91,92,95,114,116,117]

Run the below segment first to load the coloring scheme for py3Dmol

In [7]:
import py3Dmol

def display_interface(pdb_path, ab_residues, ag_residues):
    with open(pdb_path, 'r') as f:
        pdb_data = f.read()

    view = py3Dmol.view(width=800, height=500)
    view.addModel(pdb_data, 'pdb')

    # Default: cartoon and light gray
    view.setStyle({'cartoon': {'color': 'lightgray'}})

    # Highlight Ab_active (chain A, red)
    for resi in ab_residues:
        view.addStyle({'chain': 'A', 'resi': str(resi)},
                      {'cartoon': {'color': 'red'}})

    # Highlight Ag_passive (chain B, orange)
    for resi in ag_residues:
        view.addStyle({'chain': 'B', 'resi': str(resi)},
                      {'cartoon': {'color': 'orange'}})

    view.zoomTo()
    return view.show()


In the next segment, you will be able to visualize the molecules. I have used here 'cluster5_1.pdb'. You can start with this and later change other PDB files too.

Since have defined the entire surface of the antigen as passive, the actual epitope is not compelled to be at the interface. It can still happen, however this pose won’t be preferred to any other.

Are the passive residues at the interface in different clusters? How is it shown in the HADDOCK score?

In [8]:
# Replace with the cluster PDB you want to inspect
display_interface("/content/3FG005/Lab02_Docking/Scenario_1/cluster5_1.pdb", ab_active, ag_passive)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

**Scenario 2: A loose definition of the epitope is known**


Now we can compare our docking without any information about the epitope with a run where the epitope was defined.

We will proceed the same way as in Scenario 1. We downloaded the best models of individual clusters, the first cluster PDB files from the results page: https://rascar.science.uu.nl/haddock2.4/result/4242424242/0-4G6M-Ab-Ag-sas-filtered and compare them to 4G6M-matched.


In [9]:
import glob


# Adjust the path to match your repo structure
pdb_folder_2 = "/content/3FG005/Lab02_Docking/Scenario_2/"
pdb_files = sorted(glob.glob(pdb_folder_2 + "*.pdb"))

ref_file = [f for f in pdb_files if "4G6M-matched" in f][0]
cluster_files = [f for f in pdb_files if "4G6M-matched" not in f]

print("Reference:", ref_file)
print("Cluster files:", cluster_files)

Reference: /content/3FG005/Lab02_Docking/Scenario_2/4G6M-matched.pdb
Cluster files: ['/content/3FG005/Lab02_Docking/Scenario_2/cluster1_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_2/cluster2_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_2/cluster3_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_2/cluster4_1.pdb', '/content/3FG005/Lab02_Docking/Scenario_2/cluster5_1.pdb']


The following segment will provide th RMSD values. Has the L-RMSD value decreased and the CAPRI quality improved? Are the passive residues of the antigen always on the interface?

In [10]:
# Parse reference once
ref_A = parse_pdb_backbone_chain_atoms(ref_file, 'A', resid_range=(0, 225))
ref_B = parse_pdb_backbone_chain_atoms(ref_file, 'B')

for mob_file in cluster_files:
    mob_A = parse_pdb_backbone_chain_atoms(mob_file, 'A', resid_range=(0, 225))
    mob_B = parse_pdb_backbone_chain_atoms(mob_file, 'B')

    # Match atom keys
    keys_A = sorted(set(ref_A) & set(mob_A))
    keys_B = sorted(set(ref_B) & set(mob_B))

    if len(keys_A) == 0 or len(keys_B) == 0:
        print(f"{os.path.basename(mob_file)}: No common atoms — skipping.")
        continue

    P_A = np.array([mob_A[k] for k in keys_A])
    Q_A = np.array([ref_A[k] for k in keys_A])
    P_B = np.array([mob_B[k] for k in keys_B])
    Q_B = np.array([ref_B[k] for k in keys_B])

    # Align on chain A
    P_A_c = P_A - P_A.mean(axis=0)
    Q_A_c = Q_A - Q_A.mean(axis=0)
    R = kabsch(P_A_c, Q_A_c)

    # Apply rotation to chain B and calculate RMSD
    P_B_c = P_B - P_B.mean(axis=0)
    Q_B_c = Q_B - Q_B.mean(axis=0)
    P_B_aligned = P_B_c @ R
    rmsd_val = calculate_rmsd(P_B_aligned, Q_B_c)

    print(f"RMSD of {os.path.basename(mob_file)}: {rmsd_val:.3f} Å")

RMSD of cluster1_1.pdb: 6.798 Å
RMSD of cluster2_1.pdb: 13.979 Å
RMSD of cluster3_1.pdb: 22.715 Å
RMSD of cluster4_1.pdb: 1.175 Å
RMSD of cluster5_1.pdb: 6.230 Å


Below you can also visualize the interface.

In [11]:
# Replace with the cluster PDB you want to inspect
display_interface("/content/3FG005/Lab02_Docking/Scenario_2/cluster4_1.pdb", ab_active, ag_passive)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.